# 규제 선형 모델

- 다항식이 복잡해지면서 회귀계수 값이 크게 설정되고 이는 과대적합을 유발함
    - 과대적합이 일어나면 훈련데이터에서는 높은 성능을 보이지만, 테스트데이터에서는 낮은 성능을 보임
- 선형 회귀 모델의 목적은 비용함수를 최소화하는 것
- 이떄, 규제 선형 모델은 비용함수를 최소화하는데에 패널티를 적용해 회계계수의 크기를 제한함

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
from sklearn.model_selection import train_test_split


boston_df = pd.read_csv('./data/boston_housing_train.csv')

X = boston_df.drop('MEDV',axis=1)
y = boston_df['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [13]:
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error, r2_score

def evaluate_regression(y_ture, y_pred):
    mse = mean_squared_error(y_ture, y_pred)
    rmse = root_mean_squared_error(y_ture, y_pred)
    mae = mean_absolute_error(y_ture, y_pred)
    r2 = r2_score(y_ture, y_pred)

    return(f'MSE: {mse} | RMSE: {rmse} | MAE: {mae} | R2score: {r2}')

### Ridge 회귀

- L2 규제를 적용한 회귀 모델
- 회귀 계수들(가중치, 모델 파라미터)의 제곱합을 alpha 패널티를 곱해 비용에 산정
    - alpha 값이 커지면 회귀계수가 작아짐

In [14]:
from sklearn.linear_model import Ridge
reg_l2 = Ridge()

reg_l2.fit(X_train,y_train)
reg_l2.score(X_train,y_train), reg_l2.score(X_test,y_test)
evaluate_regression(y_test,reg_l2.predict(X_test))

'MSE: 22.480475501233855 | RMSE: 4.741357980709098 | MAE: 3.050375126006179 | R2score: 0.678974832784608'

In [18]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

reg_l2_model = Pipeline([
    ('poly',PolynomialFeatures(include_bias=False, degree=2)),
    ('reg_l2',Ridge(alpha=100))
])

reg_l2_model.fit(X_train,y_train)
print(reg_l2_model.score(X_train,y_train),reg_l2_model.score(X_test,y_test))
evaluate_regression(y_test,reg_l2_model.predict(X_test))

0.922557959447853 0.796056183892697


'MSE: 14.281602907949667 | RMSE: 3.7791008067991076 | MAE: 2.4589907450590327 | R2score: 0.796056183892697'

### Lasso 회귀

- L1 규제를 적용한 선형 회귀 모델
- 회귀계수 절대값의 합을 alpha 값과 곱해 규제 정도 지정
    - alpha 값이 커지면, 규제 정도가 커짐
- 회귀계수를 0까지 줄일 수 있음 == 해당 특성을 아예 반영하지 않도록 함 == 특성 선택 효과

In [ ]:
from sklearn.linear_model import Lasso
reg_l1 = Lasso(alpha=10)    # 규제 강도를 높일수록 0이 많아짐
reg_l1.fit(X_train,y_train)
print(reg_l1.score(X_train,y_train), reg_l1.score(X_test,y_test))
print(reg_l1.coef_)

0.5373948270602298 0.4945997251309123
[ 0.          0.00806625  0.          0.          0.          0.
  0.          0.          0.         -0.00896833  0.          0.01014021
 -0.62986968]


### Elastic 회귀

- L1, L2 규제를 함께 적용하는 회귀 모델
- 하이퍼 파라미터
    - alpha: 규제 정도 (L1규제 + L2규제)
    - l1_ratio: L1 규제의 비율 (L2 규제의 비율 == 1 - l1_ratio)

In [ ]:
from sklearn.linear_model import ElasticNet

reg_ela = ElasticNet(alpha=1,l1_ratio=0.7) # l1_ratio는 l1 실행 정도, 상대적으로 l2 결정
# 상대적으로 l2_ratio가 높으면 없어지는 가중치 값이 많아짐

reg_ela.fit(X_train,y_train)
print(reg_ela.score(X_train,y_train),reg_ela.score(X_test,y_test))
print(reg_ela.coef_)

0.691265488545522 0.6540836647149817
[-0.09551711  0.03198885 -0.          0.          0.          1.22175286
  0.01410072 -0.6665987   0.24640644 -0.01286154 -0.73208161  0.01197998
 -0.77912076]
